In [92]:
import pandas as pd
import geocoder
import numpy as np

In [93]:
education_df = pd.read_csv("university_df.csv")
education_df = education_df.drop("endYear", 1)
education_df[['startYear']] = education_df[['startYear']].fillna(value = 2017)

In [94]:
education_df = education_df[education_df.country.notnull()]

In [95]:
# education_df_empty_country = education_df[pd.isnull(education_df).any(axis=1)]

In [96]:
# school_country_dict = dict()
# names = education_df_empty_country['name'].unique()
# for name in names:
#     g = geocoder.google(name)
#     school_country_dict.update({name: g.country})

In [97]:
# for index, row in education_df_empty_country.iterrows():
#     country = school_country_dict.get(row['name'])
#     education_df.set_value(index, 'country', country)

# Insert actual ethnicity
# Insert type of the start year
### Type 1 > type 2 > type 3 ...
### Type 1 is farthest from now

In [98]:
def convert_actual_ethnicity(ethnicity):
    if ethnicity == 'White/Caucasian':
        return 'White'
    elif ethnicity == 'Hispanic/Latino':
        return 'Hispanic'
    elif ethnicity == 'Other/Multiracial':
        return 'Other'
    elif ethnicity == 'Black/African American':
        return 'Black'
    return ethnicity

In [99]:
education_df['actual_ethnicity'] = ''
education_df['type_year'] = ''
cand_infor_df = pd.read_csv("candilate_infor_crawed.csv")
for index, row in cand_infor_df.iterrows():
    cand_id = row['cand_id']
    actual_ethnicity = row['ethnicity']
    
    search_result = education_df[education_df.cand_id == cand_id]
    search_result = search_result.sort_values(['startYear'], ascending=True)
    
    i = 1
    for index_2, row_2 in search_result.iterrows():
        education_df.set_value(index_2, 'actual_ethnicity', convert_actual_ethnicity(actual_ethnicity))
        education_df.set_value(index_2, 'type_year', "type_"+str(i))
        i = i + 1

In [100]:
education_df = education_df[(education_df.type_year == 'type_1') \
                        | (education_df.type_year == 'type_2')\
                        | (education_df.type_year == 'type_3')]

In [101]:
# education_statistic = education_df[['type_year', 'country', 'actual_ethnicity', 'cand_id']].groupby(['type_year', 'country', 'actual_ethnicity']).agg(['count'])

In [102]:
# education_statistic.to_csv("education_statistic.csv")

# Statistic education

In [103]:
country_eth = pd.read_csv("country_ethnicity.csv", na_values=['', 'NaN'], keep_default_na = False)
country_eth.fillna(0, inplace=True)

In [104]:
education_df['predict_score'] = 0.0

In [105]:
# for index, row in education_df.iterrows():
#     country_ethinicity = \
#         country_eth[country_eth.country_code == row['country']].iloc[0]
#     education_df.set_value(index, 'predict_score', \
#                            country_ethinicity[row['actual_ethnicity']])

In [106]:
# education_df[['predict_score']] = education_df[['predict_score']].fillna(value = 0)

In [107]:
# len(education_df['cand_id'].value_counts())

In [108]:

# education_df[(education_df.type_year == 'type_2') & (education_df.predict_score > 0)].mean()

In [109]:
# education_df[(education_df.type_year == 'type_2') & (education_df.predict_score > 0)].mean()

In [110]:
# education_df[(education_df.type_year == 'type_3') & (education_df.predict_score > 0)]

In [111]:
w = [0.65, 0.2, 0.15]
education_df['predict_ethnicity'] = ''

In [112]:
names = ['White','Black','Asian','Hispanic','Other']
for cand_id in education_df.cand_id.unique():
    edu_his = education_df[education_df.cand_id == cand_id].sort_values(by="type_year")
    ethnicity_probabiblies = [0.0] * 5 # probability of each ethnicity
    i = 0
    for index, row in edu_his.iterrows():
        eth_distriubtion = country_eth[country_eth.country_code == \
                                       row['country']].iloc[0][['White','Black','Asian','Hispanic','Other']]
        for j in range(0,5):
            ethnicity_probabiblies[j] = ethnicity_probabiblies[j] + eth_distriubtion[j]
        education_df.set_value(index, 'predict_ethnicity', names[np.argmax(ethnicity_probabiblies)])
        
        
        i = i + 1
        
  

In [113]:
def change_name(ethnicity):
    if ethnicity == "Asian":
        return "Asian"
    if ethnicity == 'White':
        return "White"
    return "Other"

education_df['actual_ethnicity_2'] = education_df.apply(lambda x: change_name(x['actual_ethnicity']), axis = 1)
education_df['predict_ethnicity_2'] = education_df.apply(lambda x: change_name(x['predict_ethnicity']), axis = 1)

In [120]:
pd.crosstab(education_df.predict_ethnicity_2, education_df.actual_ethnicity_2, margins=True)

actual_ethnicity_2,Asian,Other,White,All
predict_ethnicity_2,,,,
Asian,178,22,77,277
Other,0,4,11,15
White,223,118,1757,2098
All,401,144,1845,2390


In [123]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(education_df.actual_ethnicity_2, education_df.predict_ethnicity_2, average='macro')

(0.58224339877108722, 0.47465719170907422, 0.48886260197969672, None)

In [115]:
education_df[education_df.cand_id == 0].sort_values(by="type_year")

,cand_id,country,name,startYear,actual_ethnicity,type_year,predict_score,predict_ethnicity,actual_ethnicity_2,predict_ethnicity_2
2,0,US,b'MSU',2001,White,type_1,0,White,White,White
1,0,CZ,b'ASEM',2002,White,type_2,0,White,White,White
0,0,US,b'Foothill College',2017,White,type_3,0,White,White,White


In [116]:
country_eth[country_eth.country_code == 'US']

,Unnamed: 0,country,White,Black,Asian,Hispanic,Other,country_code
183,185,United States,0.75,0.12,0.03,0.05,0.05,US


In [117]:
country_eth[country_eth.country_code == 'CZ']

,Unnamed: 0,country,White,Black,Asian,Hispanic,Other,country_code
45,45,Czech Republic,1,0,0,0,0,CZ
